# Results of DDNet Trained on CIFAR10

In this notebook I display results in terms of accuracy on test data for DDNet models trained on the CIFAR10 dataset using various regularization techniques. The DDNet model and the CIFAR10 dataset were set up as in Hoffman 2019 (for all the models). The CIFAR10 data is preprocessed by normalizing using mean [0.5, 0.5, 0.5] and variance [0.5, 0.5, 0.5]. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization with a dropout rate of p_drop = 0.5. I also train a model with Jacobian Regularization without Dropout. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 50 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_CIFAR10
from model_classes import DDNet
from tools import accuracy, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_CIFAR10()

# Summary of model
summary_model = DDNet().to(device)
summary(summary_model, (3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,792
            Conv2d-2           [-1, 64, 28, 28]          36,928
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 12, 12]          73,856
            Conv2d-5          [-1, 128, 10, 10]         147,584
         MaxPool2d-6            [-1, 128, 5, 5]               0
            Linear-7                  [-1, 256]         819,456
           Dropout-8                  [-1, 256]               0
            Linear-9                  [-1, 256]          65,792
          Dropout-10                  [-1, 256]               0
           Linear-11                   [-1, 10]           2,570
Total params: 1,147,978
Trainable params: 1,147,978
Non-trainable params: 0
---------------

In [3]:
# Load models
dataset = "cifar10"

model_names_set = [
    "model_no_reg",
    "model_l2",
    "model_jacobi",
    "model_jacobi_no_dropout",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (95%)
0,model_no_reg,51.668 +/- 2.453
1,model_l2,76.984 +/- 0.509
2,model_jacobi,52.28 +/- 5.99
3,model_jacobi_no_dropout,59.174 +/- 0.407
4,model_svb,74.57 +/- 1.306


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [4]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
# model_names = [
#     "model_no_reg_0",
#     "model_l2_0",
#     "model_jacobi_0",
#     "model_jacobi_no_dropout_0",
#     "model_svb_0",
# ]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 25/25 [00:00<00:00, 446.39it/s]


In [5]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                       0.010  \
                                   mean conf_interval          mean   
model_no_reg_0             11840.416431   1446.350781   6568.299619   
model_l2_0                  3626.486897     963.42764   5830.436304   
model_jacobi_0             12053.884611   1277.790446  13056.499424   
model_jacobi_no_dropout_0   19705.20382   1844.003387  27382.423025   
model_svb_0                 2073.162787    362.730241    886.396347   
model_no_reg_1             17590.802326   1821.741632  20132.623791   
model_l2_1                  3948.707807    934.097378   8584.765843   
model_jacobi_1             11825.606178    1397.47176  13955.632144   
model_jacobi_no_dropout_1  17004.440063   2085.575292  31326.684089   
model_svb_1                 1755.081453    336.785549   1555.244086   
model_no_reg_2             11622.148515   1674.011616  16482.461235   
model_l2_2                  3448.001554    727.241706   5311.088492   
model_jacobi_2             15968.240933   1245.972165  18522.241927   
model_jacobi_no_dropout_2  23267.480588    1746.74477  32921.943093   
model_svb_2                 2278.505288    457.196883   2709.042833   
model_no_reg_3             16347.035742   1455.470506  18598.188277   
model_l2_3                  3232.978656    745.086174   6512.041933   
model_jacobi_3             11726.237758   1569.237187   8473.545825   
model_jacobi_no_dropout_3  21122.520159    2160.00347     36972.594   
model_svb_3                 2010.423903    452.177753   2978.265668   
model_no_reg_4             15934.210154   1291.655045  17412.480785   
model_l2_4                  2442.562186    663.729834   3545.022528   
model_jacobi_4             17140.762578   1476.717476  16346.882447   
model_jacobi_no_dropout_4  18807.282374   2245.807854  28500.493994   
model_svb_4                 1746.000972    386.229244   1753.803145   

                                                0.001                
                          conf_interval          mean conf_interval  
model_no_reg_0               796.354618    889.628563     289.69644  
model_l2_0                   1050.09825  10329.029558    910.338766  
model_jacobi_0              1051.091838  15452.940854   1317.850938  
model_jacobi_no_dropout_0   1375.484542  16729.128269   1787.476866  
model_svb_0                  170.660725    324.119868    211.051087  
model_no_reg_1              1525.684448   4887.058893    947.359127  
model_l2_1                   826.756988  11695.277899   1046.460708  
model_jacobi_1               984.816915  14170.115646   1161.207009  
model_jacobi_no_dropout_1   1484.584068  27396.586585    1275.15625  
model_svb_1                  398.007516   5490.838068    385.059516  
model_no_reg_2              1452.543943  20645.598161   1664.009912  
model_l2_2                   819.310398   10609.50708    867.647835  
model_jacobi_2               928.379401  18761.961973   1417.609749  
model_jacobi_no_dropout_2    1289.64511  32399.925536   1602.896089  
model_svb_2                  583.186888   3834.976223     418.65787  
model_no_reg_3              1238.548864   7025.300103    903.678133  
model_l2_3                   860.527555  12640.742715    955.496382  
model_jacobi_3              1341.785303   1428.851934    469.572492  
model_jacobi_no_dropout_3   1428.431153  33313.950059   1958.436759  
model_svb_3                  586.057259   3814.174444    428.797217  
model_no_reg_4              1375.629738  25915.943699   1527.276744  
model_l2_4                   712.621427   7437.729917    777.715386  
model_jacobi_4              1163.402354   9463.167708   1351.931084  
model_jacobi_no_dropout_4   1633.871496  25997.116535   1644.259982  
model_svb_4                  563.212239   4424.603118    561.925989

#### Anisotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
# model_names = [
#     "model_no_reg_0",
#     "model_l2_0",
#     "model_jacobi_0",
#     "model_jacobi_no_dropout_0",
#     "model_svb_0",
# ]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 25/25 [00:00<00:00, 93.45it/s]


In [7]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                   0.010                \
                               mean conf_interval      mean conf_interval   
model_no_reg_0             14850.48   1796.013149   7571.36    984.568042   
model_l2_0                  3247.62    970.868929   6313.24   1151.343793   
model_jacobi_0             13052.28   1259.970799  14798.92   1162.553096   
model_jacobi_no_dropout_0  22043.34    2377.61859  29735.82   1976.369672   
model_svb_0                 2396.62    378.507971   1071.24    190.572126   
model_no_reg_1             19442.48   1500.791939  23517.64   1061.302329   
model_l2_1                  5393.98   1149.141554  10187.52   1004.412247   
model_jacobi_1             13268.46   1032.141221  15666.16    912.521134   
model_jacobi_no_dropout_1   19376.9   2291.164031  33820.08   1748.664139   
model_svb_1                 2061.82    448.912702   1947.32    471.585706   
model_no_reg_2              12770.7   1674.433196  18590.82   1426.236185   
model_l2_2                  3397.14     732.45093   5839.82    951.972615   
model_jacobi_2             18196.22   1530.817314  20300.44   1183.686742   
model_jacobi_no_dropout_2  26317.88   2021.295464  36653.06   1758.815062   
model_svb_2                 2658.54    587.313548    3065.5    756.600079   
model_no_reg_3             17660.72   1712.723547  21887.92   1430.627837   
model_l2_3                  2796.48    749.578526    6369.3     813.62867   
model_jacobi_3              13402.1   1789.604681   9994.52   1653.805219   
model_jacobi_no_dropout_3  23613.88   2738.531186  39642.84   1489.446911   
model_svb_3                 2071.88    362.842692   2097.54    450.276669   
model_no_reg_4             17651.06   1836.100428  21025.58   1890.087723   
model_l2_4                   2985.5    948.699111   4219.88    967.031605   
model_jacobi_4              17638.8   1610.620181  17665.52   1187.128318   
model_jacobi_no_dropout_4  21297.28   2466.698769  31375.94   1988.416574   
model_svb_4                 2072.72    392.601653    1692.8    389.213581   

                              0.001                
                               mean conf_interval  
model_no_reg_0               804.46    262.540422  
model_l2_0                  10350.1    888.003573  
model_jacobi_0             15282.12   1446.220453  
model_jacobi_no_dropout_0   19098.7   1692.725527  
model_svb_0                  413.84    230.657385  
model_no_reg_1               7283.8   1093.424411  
model_l2_1                 12996.76   1259.146308  
model_jacobi_1             15333.46   1141.649501  
model_jacobi_no_dropout_1  30986.48   1831.115359  
model_svb_1                 5781.68    391.268274  
model_no_reg_2             24135.84   1615.534139  
model_l2_2                 12733.82    1198.70685  
model_jacobi_2             17867.52   1403.378558  
model_jacobi_no_dropout_2  37217.32   1602.353027  
model_svb_2                  4849.5     599.65741  
model_no_reg_3              7685.76   1145.991551  
model_l2_3                  14762.8   1016.115445  
model_jacobi_3              1287.72    346.930759  
model_jacobi_no_dropout_3  36418.78    1774.35239  
model_svb_3                 4447.72    479.033064  
model_no_reg_4             27799.54   1702.314307  
model_l2_4                  8340.26    753.196505  
model_jacobi_4             10550.84   1373.035471  
model_jacobi_no_dropout_4  29407.54   1453.596543  
model_svb_4                 4868.46    857.743393